In [1]:
import pprint
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
import requests
import time
import os

In [2]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15'}

# Scrape the list of political subReddits

In [3]:
url="https://www.reddit.com/r/redditlists/comments/josdr/list_of_political_subreddits/"

In [6]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")
title = page_soup.find("title")
print(title)
containers = page_soup.findAll('a',class_="_3t5uN8xUmg0TOwRCOGQEcU")

names=[]
urls=[]
pairs=[]

for container in containers[:174]:
    names.append(container.text)
    urls.append(container['href'])
    pairs.append((container.text,container['href']))
    print(container.text)
    print(container['href'])

subreddit=pd.DataFrame(pairs,columns=['name','url'])


<title>List of political subreddits. : redditlists</title>
World News
http://www.reddit.com/r/worldnews
News
http://www.reddit.com/r/news
World Politics
http://www.reddit.com/r/worldpolitics
World Events
http://www.reddit.com/r/Worldevents
Business
http://reddit.com/r/business
Economics
http://reddit.com/r/Economics
Environment
http://reddit.com/r/environment
Energy
http://www.reddit.com/r/energy
Law
http://reddit.com/r/law/
Education
http://www.reddit.com/r/education/
Government
http://www.reddit.com/r/government/
History
http://www.reddit.com/r/history/
Politics PDF's
http://www.reddit.com/r/PoliticsPDFs/
Wikileaks
http://www.reddit.com/r/WikiLeaks/
SOPA
http://www.reddit.com/r/SOPA/
News Porn
http://www.reddit.com/r/NewsPorn
World News 2
http://www.reddit.com/r/worldnews2/
Anarchist News
http://www.reddit.com/r/AnarchistNews/
Republic of Politics
http://www.reddit.com/r/republicofpolitics
LGBT News
http://www.reddit.com/r/LGBTnews/
Politics 2
http://www.reddit.com/r/politics2/
Econo

In [7]:
subreddit.to_csv('Data/subReddit_list.csv')

https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

# Scrape Reddit Post

In [9]:
import requests

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('fQ-VABpboAY8iQ', 'v6X0ifFGgMRop3g2TlbV8cyX0LXxOQ')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'JoyceOoops',
        'password': '19960703jyr'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
# Add headers=headers to every request
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)


<Response [200]>

In [10]:
import praw

reddit = praw.Reddit(client_id = 'fQ-VABpboAY8iQ', 
                     client_secret = 'v6X0ifFGgMRop3g2TlbV8cyX0LXxOQ', 
                     user_agent = 'JoyceOoops')

In [11]:
for x in range(len(names))[17:]:
    
    subreddit = reddit.subreddit(urls[x].split('/')[4]) 
    posts = subreddit.hot(limit=1000)

    topics_dict = { "subreddit":[],
                   "title":[],
                   "score":[], 
                   "id":[], 
                   "url":[], 
                   "comms_num": [], 
                   "created": [], 
                   "body":[],
                   "search_type":[]}

    for submission in posts:
        topics_dict["subreddit"].append(names[x])
        topics_dict["id"].append(submission.id)
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["url"].append(submission.url)
        topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(datetime.datetime.fromtimestamp(submission.created))
    #datetime.datetime.fromtimestamp(submission.created)
        topics_dict["body"].append(submission.selftext)
        topics_dict["search_type"].append("new")

    topics_data = pd.DataFrame(topics_dict)
    topics_data.head()

    topics_data.to_csv("Data/PRAW/PRAW_"+names[x]+".csv")
    print(x,names[x])
    

Forbidden: received 403 HTTP response

In [108]:
dfs = []
docs_location = "Data/PRAW/"

for doc in os.listdir(docs_location):
    print(doc)
    df = pd.read_csv(os.path.join(docs_location + doc))
    dfs.append(df)

PRAW_Business.csv
PRAW_World News.csv
PRAW_Energy.csv
PRAW_World Events.csv
PRAW_World News 2.csv
PRAW_News.csv
PRAW_History.csv
PRAW_Economics.csv
PRAW_Politics PDF's.csv
PRAW_Environment.csv
PRAW.csv
PRAW_Government.csv
PRAW_Law.csv
PRAW_Wikileaks.csv
PRAW_News Porn.csv
PRAW_Education.csv
PRAW_World Politics.csv
PRAW_SOPA.csv


In [110]:
df=pd.concat(dfs, sort=False)
df=df.reset_index(drop=True)
df.head()

,Unnamed: 0,subreddit,title,score,id,url,comms_num,created,body,search_type
0,0,Business,Posts regarding politics,81,kurvl4,https://www.reddit.com/r/business/comments/kur...,0,2021-01-11 01:15:49,"Many of you know, we have a strict no-politics...",new
1,1,Business,USDOT report shows autonomous trucks will only...,271,m8izf6,https://www.traffictechnologytoday.com/news/au...,58,2021-03-19 15:54:04,NaN,new
2,2,Business,"NFL announces new TV deals, with Amazon gettin...",314,m8ganl,https://www.washingtonpost.com/sports/2021/03/...,67,2021-03-19 13:37:02,NaN,new
3,3,Business,Ford to hold some F-150s without certain modul...,197,m8j14u,https://www.autonews.com/manufacturing/ford-ho...,39,2021-03-19 15:56:21,NaN,new
4,4,Business,Lyft says it just had the most riders in a sin...,625,m7zhs6,https://www.cnbc.com/2021/03/18/lyft-says-it-j...,25,2021-03-18 21:09:32,NaN,new


In [111]:
df.to_csv("Data/PRAW_new.csv")

# Second method of scraping Reddit posts using Reddit Api

In [12]:
def df_from_response(res):
    # initialize dataframe
    df = pd.DataFrame()
    images=[]

# loop through each post retrieved from GET request
    for post in res.json()['data']['children']:
        # append relevant data to dataframe
        df = df.append({
            'id': post['data']['id'],
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'kind':post['kind'],
            'create_time':post['data']['created']
        }, ignore_index=True)
    
        try:
            string=post['data']['preview']['images'][0]['source']['url']
            string=string.replace('amp;', '')
            images.append(string)
        except KeyError:
            images.append(np.nan)
        
    df['image']=images
    return df

In [117]:
for x in range(len(names)):

    data = pd.DataFrame()
    params = {'limit': 100}

    for i in range(9):
    # make request
        res = requests.get(urls[x],
                           headers=headers,
                           params=params)

        # get dataframe from response
        new_df = df_from_response(res)
        # take the final row (oldest entry)
        row = new_df.iloc[len(new_df)-1]
        # create fullname
        fullname = row['kind'] + '_' + row['id']
        # add/update fullname in params
        params['after'] = fullname
    
        # append new_df to data
        data = data.append(new_df, ignore_index=True)
        print(i)
    
    data.to_csv("Data/RedditApi/RedditApi_"+names[x]+".csv")
    print(x,names[x])

KeyError: 'data'

In [119]:
urls[0]

'http://www.reddit.com/r/worldnews'

In [29]:
data

,create_time,downs,id,kind,score,selftext,subreddit,title,ups,upvote_ratio,image
0,1.616234e+09,0.0,m8wtda,t3,1.0,"Hey Guys,\n\nWhy is it that some people when u...",Python,Kmeans using Sklearn in Python Question,1.0,1.00,NaN
1,1.616233e+09,0.0,m8wkwu,t3,2.0,"Hello everyone,\n\nI'm new to this subreddit. ...",Python,Monocular Depth Estimation - Running multiple ...,2.0,1.00,https://external-preview.redd.it/RTriw7tx8Z-9y...
2,1.616230e+09,0.0,m8vqw7,t3,2.0,I now know to automate few repetitive steps us...,Python,Hosting a python script,2.0,0.75,NaN
3,1.616228e+09,0.0,m8v0ax,t3,1.0,"Its not pretty to look at, but it works.. \[*...",Python,Wiki gTTS from the terminal - 1 night project,1.0,1.00,NaN
4,1.616228e+09,0.0,m8uv8d,t3,2.0,,Python,Writing a Binary Search Tree in Python - With ...,2.0,1.00,https://external-preview.redd.it/Wv7W27i7w0Tqa...
...,...,...,...,...,...,...,...,...,...,...,...
795,1.614135e+09,0.0,lqps5m,t3,46.0,,Python,Python Developers Survey 2020 Results,46.0,0.94,https://external-preview.redd.it/PPh_JNhM8TJ-v...
796,1.614134e+09,0.0,lqpla8,t3,9.0,I will soon start my master thesis and it will...,Python,Transition to python from MATLAB,9.0,0.80,NaN
797,1.614133e+09,0.0,lqp6fu,t3,0.0,,Python,[1 day left] Full Stack Web Application Develo...,0.0,0.29,https://external-preview.redd.it/2YMaaCsqcKQic...
798,1.614133e+09,0.0,lqp43m,t3,2.0,"What ""design of experiments"" package to go wit...",Python,"What ""design of experiments"" package to go wit...",2.0,1.00,NaN


# Download images

In [17]:
import urllib.request

for x in range(len(df.title)):
    try:
        urllib.request.urlretrieve(df['image'][x], "images/"+str(x)+".jpg")
    except TypeError:
        pass

# Name Entity Recognition

In [124]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [126]:
import spacy

nlp = spacy.load("en_core_web_sm")

entities=[]

for x in df['title']:
    doc = nlp(x)
    for ent in doc.ents:
        entities.append((ent.text,ent.label_))

In [127]:
NER_lists=[]

for r in range(len(df.title)):
    doc = nlp(df.title[r])
    for ent in doc.ents:
        NER_lists.append((ent.text, ent.start_char, ent.end_char, ent.label_,df.title[r],df.id[r]))

NER=pd.DataFrame(NER_lists,columns=['TEXT','START','END','LABEL','Sentence','Article'])


In [129]:
NER.to_csv('Data/NER.csv')

In [118]:
df.to_csv('Reddit_SampleData2.csv')